<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy

## ✨ Tính năng:
- ⚡ **Multi-thread** - Copy song song (auto-detect workers)
- 🎯 **Exact match** - Kiểm tra file chính xác
- 🔄 **Auto-resume** - Tự tiếp tục khi timeout
- 🔁 **Auto-retry** - Tự retry khi rate limit
- 📊 **Progress bar** - Hiển thị tiến độ
- 🔍 **Filter** - Lọc theo đuôi file

---

## 📖 Hướng dẫn:
1. Chạy cell **Cài đặt**
2. Chạy cell **Nhập thông tin**
3. Chạy cell **Run**

## 🔥 Recommend Colab Pro+:
- GPU: **A100** | RAM cao: **BẬT**

In [ ]:
#@title 1️⃣ Cài đặt thư viện
!pip install tqdm -q
print("✅ Đã cài đặt xong!")

In [ ]:
#@title 2️⃣ Nhập thông tin
from ipywidgets import widgets, Layout
from IPython.display import display, HTML
import psutil

# Auto-detect
def detect_tier():
    try:
        ram = psutil.virtual_memory().total / (1024**3)
        if ram > 100: return 'A100 TURBO', 8, ram
        elif ram > 50: return 'A100', 6, ram
        elif ram > 20: return 'Pro+', 5, ram
        elif ram > 12: return 'Pro', 4, ram
        else: return 'Free', 2, ram
    except: return 'Unknown', 3, 0

tier, workers, ram = detect_tier()

try:
    import subprocess
    gpu = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader']).decode().strip()
except: gpu = 'No GPU'

print('🔥' + '='*48 + '🔥')
print(f'🚀 {tier.upper()} | 💾 {ram:.0f}GB RAM | 🎮 {gpu}')
print(f'⚡ Auto: {workers} workers')
print('🔥' + '='*48 + '🔥')

style = {'description_width': '150px'}
layout = Layout(width='550px')
layout_short = Layout(width='150px')

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='Link folder của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='Link folder cần copy', style=style, layout=layout)
max_size_text = widgets.Text(description='💾 Giới hạn (GB):', value='0', style=style, layout=layout_short)
workers_text = widgets.Text(description='⚡ Workers:', value=str(workers), style=style, layout=layout_short)
exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='.tmp, backup', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ đuôi:', value='', placeholder='.pdf, .mp4', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='🚫 Bỏ đuôi:', value='', placeholder='.tmp, .log', style=style, layout=layout)
dry_run_cb = widgets.Checkbox(description='🔍 Dry-run', value=False, layout=Layout(width='200px'))
overwrite_cb = widgets.Checkbox(description='📝 Ghi đè', value=False, layout=Layout(width='200px'))

display(HTML('<b>📌 Chính:</b>'))
display(dest_text, source_text)
display(HTML('<b>⚙️ Cấu hình:</b>'))
display(max_size_text, workers_text)
display(HTML('<b>🔍 Lọc:</b>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)
display(HTML('<b>🎛️ Khác:</b>'))
display(dry_run_cb, overwrite_cb)
print('✅ Nhập xong chạy cell tiếp!')

In [ ]:
#@title 3️⃣ Run - GDrive Turbo Copy
import os, time, re, json, gc
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from tqdm.notebook import tqdm

class GDriveTurboCopy:
    def __init__(self):
        self.total_mb = 0
        self.limit_gb = 0
        self.copied = 0
        self.skipped = 0
        self.errors = 0
        self.workers = 4
        self.dry_run = False
        self.overwrite = False
        self.exclude_str = []
        self.include_ext = []
        self.exclude_ext = []
        self.done_keys = set()
        self.lock = Lock()
        self.stop = False
        self.checkpoint = '/content/checkpoint.json'
        self.logfile = '/content/copy.log'
        self.svc = None

    def log(self, msg):
        ts = datetime.now().strftime('%H:%M:%S')
        print(f'[{ts}] {msg}')
        try:
            with open(self.logfile, 'a') as f: f.write(f'[{ts}] {msg}\n')
        except: pass

    def load_cp(self):
        try:
            if os.path.exists(self.checkpoint):
                with open(self.checkpoint) as f:
                    d = json.load(f)
                    self.done_keys = set(d.get('keys', []))
                    self.total_mb = d.get('mb', 0)
                    self.copied = d.get('copied', 0)
                    self.skipped = d.get('skipped', 0)
                    if self.done_keys:
                        self.log(f'✅ Resume: {len(self.done_keys)} files')
        except: pass

    def save_cp(self):
        try:
            with self.lock:
                with open(self.checkpoint, 'w') as f:
                    json.dump({'keys': list(self.done_keys), 'mb': self.total_mb, 'copied': self.copied, 'skipped': self.skipped}, f)
        except: pass

    def auth(self):
        self.log('🔐 Xác thực...')
        auth.authenticate_user()
        self.svc = build('drive', 'v3')
        self.log('✅ OK')

    def clean(self, name):
        if not name: return 'unnamed'
        for c in '\\/:*?"<>|': name = name.replace(c, '_')
        return name[:200].strip() or 'unnamed'

    def ok_ext(self, name):
        if not name: return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext: return ext in self.include_ext
        if self.exclude_ext: return ext not in self.exclude_ext
        return True

    def call(self, fn, retry=3):
        for i in range(retry):
            try: return fn()
            except HttpError as e:
                if e.resp.status == 404: return None
                if e.resp.status in [403, 429, 500, 502, 503]:
                    w = min(3 * (2**i), 30)
                    time.sleep(w)
                else: raise
            except: 
                if i < retry-1: time.sleep(3)
                else: raise
        return None

    def ls(self, fid):
        items = []
        token = None
        q = f"'{fid}' in parents and trashed=false"
        for s in self.exclude_str: q += f" and not name contains '{s}'"
        while not self.stop:
            r = self.call(lambda: self.svc.files().list(
                q=q, pageToken=token, pageSize=1000,
                fields='files(id,name,mimeType,size),nextPageToken',
                supportsAllDrives=True, includeItemsFromAllDrives=True
            ).execute())
            if not r: break
            for f in r.get('files', []):
                m = f.get('mimeType', '')
                if m == 'application/vnd.google-apps.folder':
                    items.append(f)
                elif self.ok_ext(f.get('name')) and not m.startswith('application/vnd.google-apps.'):
                    items.append(f)
            token = r.get('nextPageToken')
            if not token: break
        return items

    def exists(self, did, name, size=None):
        try:
            esc = name.replace("'", "\\'")
            r = self.call(lambda: self.svc.files().list(
                q=f"'{did}' in parents and name='{esc}' and trashed=false",
                fields='files(id,size)', supportsAllDrives=True, pageSize=1
            ).execute())
            if r and r.get('files'):
                f = r['files'][0]
                if self.overwrite and size and int(f.get('size',0)) != size:
                    try: self.svc.files().delete(fileId=f['id'], supportsAllDrives=True).execute()
                    except: pass
                    return None
                return f['id']
        except: pass
        return None

    def cp_file(self, did, src):
        if self.stop: return 'stop'
        fid = src['id']
        name = self.clean(src.get('name', 'unnamed'))
        size = int(src.get('size', 0))
        mb = size / (1024*1024)
        key = f"{did}:{fid}"

        if key in self.done_keys:
            with self.lock: self.skipped += 1
            return 'skip'

        if self.dry_run:
            self.log(f'[DRY] {name} ({mb:.1f}MB)')
            with self.lock: self.total_mb += mb
            return 'dry'

        if self.exists(did, name, size):
            with self.lock: 
                self.skipped += 1
                self.done_keys.add(key)
            return 'exists'

        try:
            t0 = time.time()
            r = self.call(lambda: self.svc.files().copy(
                body={'parents': [did], 'name': name},
                fileId=fid, supportsAllDrives=True
            ).execute())
            if r:
                spd = mb / max(time.time()-t0, 0.1)
                with self.lock:
                    self.total_mb += mb
                    self.copied += 1
                    self.done_keys.add(key)
                self.log(f'✅ {name[:50]} ({mb:.1f}MB, {spd:.0f}MB/s)')
                if self.limit_gb > 0 and self.total_mb/1024 >= self.limit_gb:
                    self.stop = True
                    return 'limit'
                return 'ok'
        except Exception as e:
            with self.lock: self.errors += 1
            self.log(f'❌ {name[:40]}: {str(e)[:50]}')
        return 'error'

    def mkdir(self, did, name):
        name = self.clean(name)
        eid = self.exists(did, name)
        if eid: return eid
        if self.dry_run: return f'dry_{name}'
        try:
            r = self.call(lambda: self.svc.files().create(
                body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [did]},
                fields='id', supportsAllDrives=True
            ).execute())
            if r:
                self.log(f'📁 {name}')
                return r['id']
        except Exception as e:
            self.log(f'❌ mkdir {name}: {e}')
        return None

    def copy_dir(self, did, sid, depth=0):
        if self.stop: return
        items = self.ls(sid)
        if not items: return

        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']

        self.log(f'{"  "*depth}📊 {len(files)} files, {len(folders)} folders')

        # Copy files với multi-thread
        if files and not self.stop:
            batch_size = 50
            for i in range(0, len(files), batch_size):
                if self.stop: break
                batch = files[i:i+batch_size]
                with tqdm(total=len(batch), desc=f'Batch {i//batch_size+1}', leave=False) as pbar:
                    with ThreadPoolExecutor(max_workers=self.workers) as ex:
                        futs = {ex.submit(self.cp_file, did, f): f for f in batch}
                        for fut in as_completed(futs):
                            pbar.update(1)
                            if fut.result() in ['limit', 'stop']:
                                self.stop = True
                                break
                self.save_cp()
                gc.collect()

        # Copy subfolders
        for folder in folders:
            if self.stop: break
            fname = folder.get('name', 'unnamed')
            self.log(f'{"  "*depth}📁 {fname}')
            sub = self.mkdir(did, fname)
            if sub and not sub.startswith('dry_'):
                self.copy_dir(sub, folder['id'], depth+1)

    def get_id(self, url):
        if not url: return None
        m = re.search(r'[-\w]{25,}', url)
        return m.group(0) if m else None

    def run(self, dest, src):
        t0 = time.time()
        self.log('='*50)
        self.log('🚀 GDRIVE TURBO COPY')
        self.log('='*50)

        self.auth()
        self.load_cp()
        self.log(f'⚙️ Workers: {self.workers} | Limit: {self.limit_gb}GB')

        did, sid = self.get_id(dest), self.get_id(src)
        if not did or not sid:
            self.log('❌ URL không hợp lệ!')
            return

        try:
            si = self.svc.files().get(fileId=sid, supportsAllDrives=True).execute()
            di = self.svc.files().get(fileId=did, supportsAllDrives=True).execute()
        except Exception as e:
            self.log(f'❌ Không truy cập được: {e}')
            return

        self.log(f'📂 Nguồn: {si.get("name")}')
        self.log(f'📂 Đích: {di.get("name")}')
        self.log('-'*50)

        new_dest = self.mkdir(did, si.get('name', 'Copy'))
        if not new_dest:
            self.log('❌ Không tạo được folder!')
            return

        self.copy_dir(new_dest, sid)
        self.save_cp()

        elapsed = time.time() - t0
        gb = self.total_mb / 1024
        spd = self.total_mb / max(elapsed, 1)

        self.log('='*50)
        self.log(f'✅ {self.copied} | ⏭️ {self.skipped} | ❌ {self.errors}')
        self.log(f'💾 {gb:.2f}GB | ⏱️ {int(elapsed)}s | 🚀 {spd:.1f}MB/s')
        self.log('='*50)
        self.log('🎉 XONG!' if self.errors == 0 else f'⚠️ {self.errors} lỗi')

# === RUN ===
print("🔧 Khởi tạo...")
dl = GDriveTurboCopy()
dl.limit_gb = float(max_size_text.value) if max_size_text.value else 0
dl.workers = max(1, min(10, int(workers_text.value))) if workers_text.value else 4
dl.exclude_str = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [('.' + s.strip().lstrip('.').lower()) for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [('.' + s.strip().lstrip('.').lower()) for s in exclude_ext_text.value.split(',') if s.strip()]
dl.dry_run = dry_run_cb.value
dl.overwrite = overwrite_cb.value

if not dest_text.value.strip(): print("❌ Nhập folder đích!")
elif not source_text.value.strip(): print("❌ Nhập folder nguồn!")
else: dl.run(dest_text.value.strip(), source_text.value.strip())

In [ ]:
#@title 4️⃣ Xóa checkpoint (chạy lại từ đầu)
import os
for f in ['/content/checkpoint.json', '/content/copy.log']:
    if os.path.exists(f): os.remove(f); print(f'✅ Xóa {f}')
    else: print(f'⏭️ Không có {f}')
print('🔄 OK!')